Try ebedding, conv, LSTM, as well as Word2Vec and Glove for movie review prediction

In [0]:
!pip install -q keras
!pip install --upgrade gensim

    100% |████████████████████████████████| 23.6MB 1.5MB/s 
    100% |████████████████████████████████| 1.4MB 15.1MB/s 
    100% |████████████████████████████████| 133kB 26.7MB/s 
    100% |████████████████████████████████| 61kB 23.2MB/s 
    100% |████████████████████████████████| 4.8MB 6.4MB/s 
    100% |████████████████████████████████| 552kB 24.8MB/s 
  Running setup.py bdist_wheel for smart-open ... - done
  Stored in directory: /root/.cache/pip/wheels/23/00/44/e5b939f7a80c04e32297dbd6d96fa3065af89ecf57e2b5f89f
  Running setup.py bdist_wheel for bz2file ... - done
  Stored in directory: /root/.cache/pip/wheels/81/75/d6/e1317bf09bf1af5a30befc2a007869fa6e1f516b8f7c591cb9
Successfully built smart-open bz2file


In [0]:
!wget http://www.cs.cornell.edu/people/pabo/movie-review-data/review_polarity.tar.gz
!tar -xvzf review_polarity.tar.gz

In [0]:
import nltk
nltk.download('stopwords')
from string import punctuation
from os import listdir
from collections import Counter
from nltk.corpus import stopwords
from string import punctuation


import numpy as np
from numpy import array,asarray,zeros 

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Dropout,Flatten,Embedding 

from keras.layers.convolutional import Conv1D,MaxPooling1D
 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Define Vocabulary

In [0]:
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text
 
# turn a doc into clean tokens
def clean_doc(doc):
	tokens = doc.split()
	table = str.maketrans('', '', punctuation)
	tokens = [w.translate(table) for w in tokens]
	tokens = [word for word in tokens if word.isalpha()]
	stop_words = set(stopwords.words('english'))
	tokens = [w for w in tokens if not w in stop_words]
	tokens = [word for word in tokens if len(word) > 1]
	return tokens
 
# load doc and add to vocab
def add_doc_to_vocab(filename, vocab):
	doc = load_doc(filename)
	tokens = clean_doc(doc)
	vocab.update(tokens)

# load all docs in a directory
def process_docs(directory, vocab, is_trian):
	for filename in listdir(directory):
		if is_trian and filename.startswith('cv9'):
			continue
		if not is_trian and not filename.startswith('cv9'):
			continue
		path = directory + '/' + filename
		add_doc_to_vocab(path, vocab)

# define vocab
vocab = Counter()

# add all docs to vocab
process_docs('txt_sentoken/neg', vocab, True)
process_docs('txt_sentoken/pos', vocab, True)

print(len(vocab))
print(vocab.most_common(50))

44276
[('film', 7983), ('one', 4946), ('movie', 4826), ('like', 3201), ('even', 2262), ('good', 2080), ('time', 2041), ('story', 1907), ('films', 1873), ('would', 1844), ('much', 1824), ('also', 1757), ('characters', 1735), ('get', 1724), ('character', 1703), ('two', 1643), ('first', 1588), ('see', 1557), ('way', 1515), ('well', 1511), ('make', 1418), ('really', 1407), ('little', 1351), ('life', 1334), ('plot', 1288), ('people', 1269), ('could', 1248), ('bad', 1248), ('scene', 1241), ('movies', 1238), ('never', 1201), ('best', 1179), ('new', 1140), ('scenes', 1135), ('man', 1131), ('many', 1130), ('doesnt', 1118), ('know', 1092), ('dont', 1086), ('hes', 1024), ('great', 1014), ('another', 992), ('action', 985), ('love', 977), ('us', 967), ('go', 952), ('director', 948), ('end', 946), ('something', 945), ('still', 936)]


In [0]:
#save vocab to a file
min_occurane = 2
tokens = [k for k,c in vocab.items() if c >= min_occurane]
print(len(tokens))

def save_list(lines, filename):
	data = '\n'.join(lines)
	file = open(filename, 'w')
	file.write(data)
	file.close()
  
save_list(tokens, 'vocab_movie.txt')

25767


In [0]:
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text
 
# turn a doc into clean tokens
def clean_doc(doc, vocab):
	# split into tokens by white space
	tokens = doc.split()
	# remove punctuation from each token
	table = str.maketrans('', '', punctuation)
	tokens = [w.translate(table) for w in tokens]
	# filter out tokens not in vocab
	tokens = [w for w in tokens if w in vocab]
	tokens = ' '.join(tokens)
	return tokens
 
# load all docs in a directory
def process_docs(directory, vocab, is_trian):
	documents = list()
	# walk through all files in the folder
	for filename in listdir(directory):
		# skip any reviews in the test set
		if is_trian and filename.startswith('cv9'):
			continue
		if not is_trian and not filename.startswith('cv9'):
			continue
		# create the full path of the file to open
		path = directory + '/' + filename
		# load the doc
		doc = load_doc(path)
		# clean doc
		tokens = clean_doc(doc, vocab)
		# add to list
		documents.append(tokens)
	return documents
 


In [0]:
# load the vocabulary
vocab_filename = 'vocab_movie.txt'
vocab = load_doc(vocab_filename)
vocab = vocab.split()
vocab = set(vocab)

# load all training reviews
train_positive_docs = process_docs('txt_sentoken/pos', vocab, True)
train_negative_docs = process_docs('txt_sentoken/neg', vocab, True)
train_docs = train_negative_docs + train_positive_docs

# create the tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_docs)
test_encoded_docs = tokenizer.texts_to_sequences(train_docs)
max_length = max([len(s.split()) for s in train_docs])
Xtrain = pad_sequences(test_encoded_docs, maxlen=max_length, padding='post')
ytrain = array([0 for _ in range(len(train_negative_docs))] + [1 for _ in range(len(train_positive_docs))])


# load all test reviews
test_positive_docs = process_docs('txt_sentoken/pos', vocab, False)
test_negative_docs = process_docs('txt_sentoken/neg', vocab, False)
test_docs = test_negative_docs + test_positive_docs
test_encoded_docs = tokenizer.texts_to_sequences(test_docs) #reuse the tokens from training set
Xtest = pad_sequences(test_encoded_docs, maxlen=max_length, padding='post')
ytest = array([0 for _ in range(len(test_negative_docs))] + [1 for _ in range(len(test_positive_docs))])

In [0]:
vocab_size = len(tokenizer.word_index) + 1

# define model
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_length))
model.add(Conv1D(filters=32, kernel_size=8, activation='relu'))
model.add(Dropout(0.4))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(Xtrain, ytrain, epochs=5, verbose=2)
# evaluate
loss, acc = model.evaluate(Xtest, ytest, verbose=0)
print('Test Accuracy: %f' % (acc*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 1317, 100)         2576800   
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 1310, 32)          25632     
_________________________________________________________________
dropout_1 (Dropout)          (None, 1310, 32)          0         
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 655, 32)           0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 655, 32)           0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 20960)             0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                209610    
__________

In [0]:
test_positive_docs[1]

'director dominic made highly underrated kalifornia producer jerry bruckheimer rock armageddon bring us slick entertaining remake film name absolutely one ever seen nicolas cage plays memphis retired car thief whos pulled back business evil car thief christopher eccleston determined kill memphis kid brother giovanni ribisi memphis ordered steal cars four days time brother meet unfortunate demise elude detectives hot trail rival car thief feels job given gang memphis sets put old crew back together discovers retired well gone sixty seconds things right opening credits sequence get rockin little tune moby along simple back story told photographs assorted objects filmmakers sometimes make break film opening title sequence one easily gets mindset entertaining ride follows doesnt disappoint cage turns one good performances easily go either way good bad dont know mannerisms dialogue delivery carry film along nicely supporting cast members angelina jolie memphis former love film provide eye c

In [0]:
model.predict_classes(Xtest)[1]

array([0], dtype=int32)

In [0]:
ytest[1]

0

# Word Embedding

A word embedding is an way to represent words using a vctor representation (not like One-hot encoding, but they have same purpose)

The values in the embedding vector are learned from text and ware based on the surrounding words. Two popular examples of embeddings are: Word3Vec and Glove. Two popular methods to train the embeddings are bag-of-word and skip-gram. Please refer to the Stanford Natural Language Processing course for details, which is the best course that I found. http://web.stanford.edu/class/cs224n/

Keras offers embedding layers (https://keras.io/layers/embeddings/#embedding) that can be easily added to your neural networks. 

keras.layers.Embedding(input_dim, output_dim, embeddings_initializer='uniform', 
                       embeddings_regularizer=None, activity_regularizer=None, 
                       embeddings_constraint=None, mask_zero=False, input_length=None)

The input of this embedding layer is required to be integer encoded (liker one-hot encode). This preparation step is usually conducted by Tokenizer, which is also provided in Keras.


![](https://qph.fs.quoracdn.net/main-qimg-3e812fd164a08f5e4f195000fecf988f)

# Word2Vec

The basic idea of Word2Vec or Glove is very simple. They uses existing corpus to generate/training the embeddings, because we agree that "the meaning of a word can be inferred by the company it keeps" .  This is similar to the transfer learning in computer vision. We saved the weights (embeddings) from pre-trained networks and than add a few new layers to model the current problem.

The word2vec algorithms include skip-gram and CBOW models, using either hierarchical softmax or negative sampling

In [0]:
from gensim.models import Word2Vec


In [0]:
model = Word2Vec(train_docs, size=100, window=5, workers=8, min_count=1)
# summarize vocabulary size in model
words = list(model.wv.vocab)
print('Vocabulary size: %d' % len(words))

# save model in ASCII (word2vec) format
filename = 'embedding_word2vec.txt'
model.wv.save_word2vec_format(filename, binary=False)

Vocabulary size: 27


In [0]:
!ls

embedding_word2vec.txt	review_polarity.tar.gz	txt_sentoken
poldata.README.2.0	sample_data		vocab_movie.txt


In [0]:
cat embedding_word2vec.txt

27 100
  -0.054297768 0.37111235 0.39346772 -0.059841778 0.2975644 -0.087068796 -0.38315597 0.0013162381 -0.119920604 0.08157335 0.15997213 0.26021948 -0.1754535 0.12761211 0.13447402 -0.02233237 -0.061185177 -0.14506665 -0.13332619 -0.08910539 -0.29466495 0.043603275 0.009737978 -0.067909405 0.32747728 -0.082866676 -0.071710125 0.058093958 -0.025532877 -0.107163884 -0.18632796 0.15541753 0.14906631 0.14483017 0.30382589 0.0378381 0.016284432 0.06754286 0.09453005 -0.11837328 -0.16858798 -0.30822954 0.26654914 -0.28213292 -0.018136185 -0.005222439 0.0047721737 0.100159906 0.17439094 -0.51919156 0.12199433 -0.33687064 -0.42350942 -0.28219742 -0.33948904 0.05306033 0.118537605 0.15010369 -0.004340681 -0.30583522 -0.43255037 0.016526617 -0.17005607 -0.36167505 -0.13703804 -0.05937956 -0.23050828 -0.097290546 -0.10880392 0.09241034 0.07001113 0.3864962 -0.018560275 0.33691743 -0.018140696 0.21017286 -0.15074301 -0.41392353 -0.0071451915 0.15077442 0.027798504 0.082764216 0.13520975 0.37295

In [0]:
def load_embedding(filename):
	# load embedding into memory, skip first line
	file = open(filename,'r')
	lines = file.readlines()[1:]
	file.close()
	# create a map of words to vectors
	embedding = dict()
	for line in lines:
		parts = line.split()
		# key is string word, value is numpy array for vector
		embedding[parts[0]] = asarray(parts[1:], dtype='float32')
	return embedding

# create a weight matrix for the Embedding layer from a loaded embedding
def get_weight_matrix(embedding, vocab):
	# total vocabulary size plus 0 for unknown words
	vocab_size = len(vocab) + 1
	# define weight matrix dimensions with all 0
	weight_matrix = zeros((vocab_size, 100))
	# step vocab, store vectors using the Tokenizer's integer mapping
	for word, i in vocab.items():
		weight_matrix[i] = embedding.get(word)
	return weight_matrix

In [0]:
#load embedding from file
raw_embedding = load_embedding('embedding_word2vec.txt')
# get vectors in the right order
embedding_vectors = get_weight_matrix(raw_embedding, tokenizer.word_index)
# create the embedding layer
embedding_layer = Embedding(vocab_size, 100, weights=[embedding_vectors], input_length=max_length, trainable=False)

# define model
model = Sequential()
model.add(embedding_layer)
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
print(model.summary())
# compile network
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model.fit(Xtrain, ytrain, epochs=10, verbose=2)
# evaluate
loss, acc = model.evaluate(Xtest, ytest, verbose=0)
print('Test Accuracy: %f' % (acc*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 1317, 100)         2576800   
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 1313, 128)         64128     
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 656, 128)          0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 83968)             0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 83969     
Total params: 2,724,897
Trainable params: 148,097
Non-trainable params: 2,576,800
_________________________________________________________________
None
Epoch 1/10
 - 17s - loss: 8.0942 - acc: 0.4978
Epoch 2/10
 - 17s - loss: 8.0590 - acc: 0.5000
Epoch 3/10
 - 17s - loss: 8

#Try Glove embeddings

In [0]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip 

--2018-11-06 19:23:20--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2018-11-06 19:23:20--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  97.1MB/s    in 8.8s    

2018-11-06 19:23:29 (93.4 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [0]:
# load the vocabulary
vocab_filename = 'vocab_movie.txt'
vocab = load_doc(vocab_filename)
vocab = vocab.split()
vocab = set(vocab)

# load all training reviews
train_positive_docs = process_docs('txt_sentoken/pos', vocab, True)
train_negative_docs = process_docs('txt_sentoken/neg', vocab, True)
train_docs = train_negative_docs + train_positive_docs

# create the tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_docs)
test_encoded_docs = tokenizer.texts_to_sequences(train_docs)
max_length = max([len(s.split()) for s in train_docs])
Xtrain = pad_sequences(test_encoded_docs, maxlen=max_length, padding='post')
ytrain = array([0 for _ in range(len(train_negative_docs))] + [1 for _ in range(len(train_positive_docs))])


# load all test reviews
test_positive_docs = process_docs('txt_sentoken/pos', vocab, False)
test_negative_docs = process_docs('txt_sentoken/neg', vocab, False)
test_docs = test_negative_docs + test_positive_docs
test_encoded_docs = tokenizer.texts_to_sequences(test_docs) #reuse the tokens from training set
Xtest = pad_sequences(test_encoded_docs, maxlen=max_length, padding='post')
ytest = array([0 for _ in range(len(test_negative_docs))] + [1 for _ in range(len(test_positive_docs))])

In [0]:
vocab_size = len(tokenizer.word_index) + 1
 
# load embedding from file
raw_embedding = load_embedding('glove.6B.100d.txt')
embedding_vectors = get_weight_matrix(raw_embedding, tokenizer.word_index)
embedding_layer = Embedding(vocab_size, 100, weights=[embedding_vectors], input_length=max_length, trainable=False)
 
# define model
model = Sequential()
model.add(embedding_layer)
model.add(Dropout(0.2))
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.summary()

# compile network
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(Xtrain, ytrain, epochs=10, verbose=2)

# evaluate
loss, acc = model.evaluate(Xtest, ytest, verbose=0)
print('Test Accuracy: %f' % (acc*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (None, 1317, 100)         2576800   
_________________________________________________________________
dropout_17 (Dropout)         (None, 1317, 100)         0         
_________________________________________________________________
conv1d_14 (Conv1D)           (None, 1313, 128)         64128     
_________________________________________________________________
max_pooling1d_12 (MaxPooling (None, 656, 128)          0         
_________________________________________________________________
dropout_18 (Dropout)         (None, 656, 128)          0         
_________________________________________________________________
flatten_12 (Flatten)         (None, 83968)             0         
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 83969     
Total para

In [0]:
print (model.predict_classes(Xtest)[1])
print (ytest[1])

[0]
0


In [0]:
train_docs[1]

'following review contains spoilers please someone stop joel schumacher vomits film tries pass movie chuck dowling jacksonville film journal wrote sentence review batman robin apparently one heard words oh wish heard oh wish someone stopped hes done plot similar paul schrader film hardcore surprisingly decent engrossing tale fact everything opening minutes see private detective tom welles nicolas cage working case prestigious member government keep fact mind cage trusted important case solves returns home wife new daughter even finds time rake yard keep mind important well hes hired rich elderly woman solve mystery husbands death comes across film private safe appears snuff film sort urban legend pornography industry someone actually brutally murdered camera woman wants welles locate young girl film hopefully disprove fact snuff film sets journey sordid underworld sleaze discover truth film main problem whole film completely pointless welles actually selected case old womans lawyer ant